# Demo
## Подготовительные работы
Откройте данный ноутбук в Google Colab, выполняйте ячейки ниже:
Клонируем git репозиторий:

In [1]:
!git clone https://github.com/nadushonkin/hifi_gan

Cloning into 'hifi_gan'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 184 (delta 61), reused 144 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (184/184), 58.64 KiB | 14.66 MiB/s, done.
Resolving deltas: 100% (61/61), done.


Создаем и активируем новую среду (после выполнения двух ячеек ниже сеанс перезапустится, это ок, просто запускайте ячейки дальше)

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


In [3]:
!conda create -n hifi_env python=3.9 -y -q
!source /usr/local/etc/profile.d/conda.sh && conda activate hifi_env

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/hifi_env

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.11.12 |       hbd8a1cb_0         149 KB  conda-forge
    ld_impl_linux-64-2.45      |default_hbd61a6d_104         709 KB  conda-forge
    libexpat-2.7.3             |       hecca717_0          75 KB  conda-forge
    libffi-3.5.2               |       h9ec8514_0          56 KB  conda-forge
    libgcc-15.2.0              |      he0feb66_16        1018 KB  conda-forge
    libgcc-ng-15.2.0           |      h69a702a_16          27 KB  conda-forge
    libgomp-15.2.0             |      he0feb66_16         589 KB  conda-forge
    liblzma-5.8.1             

Устанавливаем необходимые библиотеки:

In [1]:
!pip install -r /content/hifi_gan/requirements.txt --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 141.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 142.2 MB/s eta 0:00:00

Скачаем датасет (вставьте ссылку на него в выделенное место, по умолчанию там ссылка на test_data для тестирования из канала):

In [2]:
import requests
import zipfile
import io
import os
from urllib.parse import urlencode

def download_and_extract_from_yandex_disk(public_url, extract_to='/content/dataset'):
    base_api_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_api_url + urlencode({'public_key': public_url})

    response = requests.get(final_url)

    if response.status_code != 200:
        raise Exception(f"Не удалось получить ссылку для скачивания. Код ошибки: {response.status_code}")

    download_url = response.json()['href']

    download_response = requests.get(download_url)

    os.makedirs(extract_to, exist_ok=True)

    with zipfile.ZipFile(io.BytesIO(download_response.content)) as zip_ref:
        zip_ref.extractall(extract_to)

    absolute_path = os.path.abspath(extract_to)
    return absolute_path

# Вставьте сюда ссылку на датасет:
yandex_disk_url = "https://disk.360.yandex.ru/d/Mzh9T9OEZKSsww"

try:
    dataset_path = download_and_extract_from_yandex_disk(yandex_disk_url)
except Exception as e:
    print(f"Произошла ошибка: {e}")

Получим ссылку на папку с данными:

In [3]:
contents = os.listdir(dataset_path)


data_path = os.path.join(dataset_path, contents[0])

In [4]:
data_path

'/content/dataset/test_data'

Скачаем чекпоинт модели и сохраним на него ссылку:

In [5]:
import gdown
import os

url = "https://drive.google.com/file/d/1ibjHvTtlUEMTqVeecTmbjcXOxEr-rYXs/view?usp=sharing"

output = "model_weights.pth"

gdown.download(url, output, fuzzy=True)

checkpoint_weights_path = os.path.abspath(output)

Downloading...
From (original): https://drive.google.com/uc?id=1ibjHvTtlUEMTqVeecTmbjcXOxEr-rYXs
From (redirected): https://drive.google.com/uc?id=1ibjHvTtlUEMTqVeecTmbjcXOxEr-rYXs&confirm=t&uuid=8763b45b-72cd-496b-b772-cf2ff9e29946
To: /content/model_weights.pth
100%|██████████| 867M/867M [00:16<00:00, 51.4MB/s]


Добавим возможность подсчета WVMOS (скачаем с гита и импортируем и объявим ее, чтобы использовать дальше)

In [23]:
import sys
import torch

!{sys.executable} -m pip install git+https://github.com/AndreevP/wvmos --quiet

_original_load = torch.load
def _unsafe_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_load(*args, **kwargs)

torch.load = _unsafe_load

from wvmos import get_wvmos
model_wvmos = get_wvmos(cuda=True)

torch.load = _original_load

  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

## Продемонстрируем использование synthesize.py
### Для датасета

Предполагается, что папка с .wav находится внутри NameOfTheDirectoryWithUtterances (которую скачивали с яндекс диска выше) и называется 'gt_audio' (как в датасете для теста), если это не так, передайте ее отдельно:

datasets.test.audio_dir="путь до вашей папки"

In [15]:
!python /content/hifi_gan/synthesize.py \
                  inferencer.from_pretrained={checkpoint_weights_path} \
                  dataloader.batch_size=1 \
                  datasets.test.data_dir={data_path} \
                  datasets.test.resynthesize=True \
                  inferencer.log=False

/usr/local/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/hifi_gan/synthesize.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case 

In [25]:
mos = model_wvmos.calculate_dir("/content/hifi_gan/result", mean=True)
print(f"\nСреднее значение WVMOS для тестового датасета: {mos}")

100%|██████████| 5/5 [00:00<00:00, 32.44it/s]


Среднее значение WVMOS для тестового датасета: 1.6414606422185898


### Для текстового ввода с консоли:
можете заменить текст в параметре query, результат генерации можно найти по пути: /content/hifi_gan/result/cli_query_result.wav

In [32]:
!python /content/hifi_gan/synthesize.py \
                  --config-name=query \
                  inferencer.from_pretrained={checkpoint_weights_path} \
                  query="'Hello, this is a test of the query.'" \
                  inferencer.log=False

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/entrypoints.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [33]:
mos = model_wvmos.calculate_one("/content/hifi_gan/result/cli_query_result.wav")
print(f"Значение WVMOS для текстового запроса: {mos}")

Значение WVMOS для текстового запроса: 4.319073677062988
